# setup

In [13]:
base_url="http://host.docker.internal:11434"
model = "gemma3:27b"
from langchain_ollama import ChatOllama
model = ChatOllama(
    model=model,
    base_url=base_url,
)

# tools

In [59]:
from pydantic import create_model
import inspect
from functools import wraps

def register_function_schema(func):
    sig = inspect.signature(func)

    # Build Pydantic model dynamically from function parameters
    fields = {
        name: (param.annotation, ...)
        for name, param in sig.parameters.items()
    }

    ParamsModel = create_model(f"{func.__name__.title()}Params", **fields)

    # Attach schema to the function
    func._schema = {
        "name": func.__name__,
        "description": func.__doc__,
        "parameters": ParamsModel.model_json_schema()
    }

    return func

In [73]:
from langchain_core.tools import tool


@register_function_schema
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@register_function_schema
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

In [198]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

AttributeError: 'function' object has no attribute 'name'

In [168]:
r = json.dumps([tool._schema for tool in tools], indent=4)

In [195]:
system_prompt = f"""\
You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of
{{{{"name": function name, "parameters": dictionary of argument name and its value}}}}

You SHOULD NOT include any other text in the response if you call a function
{rendered_tools}
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [196]:
print(system_prompt)

You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of
{{"name": function name, "parameters": dictionary of argument name and its value}}

You SHOULD NOT include any other text in the response if you call a function
multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.



In [197]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'parameters': {'x': 13.0, 'y': 4.0}}

In [124]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [90]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

```json
{
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}
```


In [104]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4 and 12 + 7"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

In [25]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

In [28]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

In [29]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

In [30]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

In [31]:
from langchain.agents import create_tool_calling_agent

In [58]:
@register_function_schema
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y

# Access the schema
add._schema

{'name': 'add',
 'description': 'Add two numbers.',
 'parameters': {'properties': {'x': {'title': 'X', 'type': 'integer'},
   'y': {'title': 'Y', 'type': 'integer'}},
  'required': ['x', 'y'],
  'title': 'AddParams',
  'type': 'object'}}